In [1]:
# the purpose of this notebook is to explore the separated 10m, 30m, and 50m datasets and their aggregate data (average
# pitch/roll, variance in turbulence, battery loss, all that jazz)

In [32]:
# import tings
import pandas as pd
import numpy as np
from pathlib import Path

In [33]:
# fix individual logs
# running the function(s) below, I found that Jan07.3-35_10m and Jan06.4-47_30m had mistakenly collected data on the way DOWN, too, which confounds
# the data. here, we reassign that file itself but without the distracting irrelevant data

badfile1 = pd.read_csv(Path('UsableLogs/Jan07.3-35_10m.csv'))
badfile1 = badfile1[badfile1['index'] <= 381]
badfile1.to_csv('UsableLogs/Jan07.3-35_10m.csv', index=False)

badfile2 = pd.read_csv(Path('UsableLogs/Jan06.4-47_30m.csv'))
badfile2 = badfile2[badfile2['index'] <= 603]
badfile2.to_csv('UsableLogs/Jan06.4-47_30m.csv', index=False)

In [40]:
# function that turns the files fed into it into a single row of aggregate data

# if file ends with _10m, entry in 'altitude' section is 10m, etc.
# include file name to check weird outliers out
# find average of all speeds, voltage, current, pitch, roll, battery_percent lost over time
# each time, append the row to the dataframe previously made

directory = Path('UsableLogs')
preaggregate_logs = directory.glob('*.csv')
output_dir = Path('AggregateLogs')

def process_usable_logs():
    aggregated_data = []

    for file_path in preaggregate_logs:        
        # Parse altitude from filename
        altitude = None
        if '_10m' in file_path.name:
            altitude = 10
        elif '_30m' in file_path.name:
            altitude = 30
        elif '_50m' in file_path.name:
            altitude = 50
        
        try:
            # Read the CSV
            df = pd.read_csv(file_path)
            start_time, end_time = pd.to_datetime(df['timestamp'].iloc[0]), pd.to_datetime(df['timestamp'].iloc[-1])
            flight_time = (end_time - start_time).total_seconds()
            df['tilt_magnitude'] = np.sqrt(df['pitch(degrees)']**2 + df['roll(degrees)']**2)
            df['power(watt)'] = df['voltage(v)'] * df['current(A)']
            
            # calculate the aggregate stats
            stats = {
                'file_name': file_path.stem,
                'flight_time(seconds)': flight_time,
                'altitude(m)': altitude,
                'speed_avg': df['speed(mph)'].mean(),
                'speed_sd': df['speed(mph)'].std(),
                'power_avg': df['power(watt)'].mean(),
                'power_sd': df['power(watt)'].std(),
                'tilt_avg': df['tilt_magnitude'].mean(),
                'tilt_sd': df['tilt_magnitude'].std(),
                'turbulence_intensity': df['tilt_magnitude'].std() / (df['tilt_magnitude'].mean() + 0.05) # + 0.05 to avoid div by zero

            }
            
            aggregated_data.append(stats)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {e}")

    return pd.DataFrame(aggregated_data).sort_values('file_name')
aggregate_df = process_usable_logs()

In [43]:
# function that turns the files fed into it into a several more rows of data to feed to the random forest model.
# uses a sliding window to collect data at every 5 second interval (only moving forward 0.5 seconds per step, leading
# to more overlap for more thorough data). naming conventions and other data collected is otherwise similar

directory = Path('UsableLogs')
preaggregate_logs = directory.glob('*.csv')
output_dir = Path('')

def apply_window_to_usable_logs():
    windowed_data = []

    for file_path in preaggregate_logs:        
        # Parse altitude from filename
        altitude = None
        if '_10m' in file_path.name:
            altitude = 10
        elif '_30m' in file_path.name:
            altitude = 30
        elif '_50m' in file_path.name:
            altitude = 50
        
        try:
            # Read the CSV
            df = pd.read_csv(file_path)
            start_time, end_time = pd.to_datetime(df['timestamp'].iloc[0]), pd.to_datetime(df['timestamp'].iloc[-1])
            flight_time = (end_time - start_time).total_seconds()
            df['tilt_magnitude'] = np.sqrt(df['pitch(degrees)']**2 + df['roll(degrees)']**2)
            df['power(watt)'] = df['voltage(v)'] * df['current(A)']

            WINDOW_SIZE = 50 # cut up the data every 5 seconds (10Hz)
            STEP_SIZE = 5 # 0.5 second overlap so all gusts can be caught (moving the window 0.5 seconds at a time)

            for i in range(0, len(df) - WINDOW_SIZE, STEP_SIZE):
                window = df.iloc[i:i + WINDOW_SIZE]
                # calculate the aggregate stats
                stats = {
                    'file_name': file_path.stem,
                    'curr_step': i,
                    'file_flight_time(seconds)': flight_time,
                    'altitude(m)': altitude,
                    'speed_avg': window['speed(mph)'].mean(),
                    'speed_sd': window['speed(mph)'].std(),
                    'power_avg': window['power(watt)'].mean(),
                    'power_sd': window['power(watt)'].std(),
                    'tilt_avg': window['tilt_magnitude'].mean(),
                    'tilt_sd': window['tilt_magnitude'].std(),
                    # how 'gusty' is the wind relative to its base speed (is the drone tilting crazily right now compared to how it is on average)
                    'turbulence_intensity': window['tilt_magnitude'].std() / (window['tilt_magnitude'].mean() + 0.05) # + 0.05 to avoid div by zero
                }
                windowed_data.append(stats)
            
        except Exception as e:
            print(f"Error processing {file_path.name}: {e}")

    return pd.DataFrame(windowed_data).sort_values('file_name')
windowed_df = apply_window_to_usable_logs()

In [46]:
# push to csv
aggregate_df.to_csv('aggregate_data.csv', index=False) 
windowed_df.to_csv('windowed_data.csv', index=False) 

In [37]:
aggregate_df.groupby('altitude(m)')[['tilt_avg']].mean()

,tilt_avg
altitude(m),
10,5.417782
30,8.934385
50,10.015134


In [29]:
# seeing which values matter most to determining altitude
# inititally these results look a little disappointing, but I have a theory that the wind shear is not linear.
# We'll use a random forest decision tree to get the real pattern out of these variables in another notebook.

aggregate_df.drop(columns={'file_name', 'flight_time(seconds)'}).corr()['altitude(m)'].sort_values(ascending=False)

altitude(m)                1.000000
tilt_avg                   0.524320
current_sd                 0.208752
z_speed_avg                0.142879
voltage_sd                 0.098318
battery_loss_per_second    0.082652
y_speed_avg                0.018275
current_avg                0.010352
x_speed_avg                0.003482
speed_avg                 -0.037920
speed_sd                  -0.040284
compass_heading_avg       -0.061019
compass_heading_std       -0.067177
tilt_sd                   -0.084703
voltage_avg               -0.152381
Name: altitude(m), dtype: float64